In [88]:

import nbimporter

from Database_connection.database_connection import connect_db
import requests
import pandas as pd
import numpy as np
from datetime import datetime

# Rest of your code...



In [80]:
response = requests.get('https://api.llama.fi/protocols')
data = response.json()
df = pd.DataFrame.from_dict(data, orient = 'Columns')

In [82]:
df = df [[
       'id', 'name', 'address', 'symbol', 'url', 'description', 'chain',
       'logo', 'audits', 'gecko_id', 'category',
#        'chains', 
    'module', 'twitter', 'listedAt',
       'slug', 'tvl',
#     'chainTvls', 'forkedFrom', 'oracles', 
    'change_1h', 'change_1d', 'change_7d',
       # 'tokenBreakdowns', 
]]

In [75]:
# df = df[['id', 'name', 'address', 'symbol', 'url', 'description', 'chain',
#        'logo', 'audits', 'gecko_id', 'cmcId', 'category',
#        'module', 'twitter', 'listedAt',
#        'slug', 'tvl', 
#        # 'chainTvls','chains','tokenBreakdowns', 'forkedFrom', 'oracles' Creates troble cuase its a dict 
#        'change_1h', 'change_1d', 'change_7d',
#        'mcap', 'treasury', 'audit_links',
#        'governanceID', 'github', 'parentProtocol',
#        'staking', 'pool2']]

KeyError: "['cmcId', 'mcap', 'treasury', 'audit_links', 'governanceID', 'github', 'parentProtocol', 'staking', 'pool2'] not in index"

In [84]:
df.isna().sum().sort_values(ascending = False)

# df.columns

gecko_id       2051
address        1119
change_7d       724
listedAt        637
change_1d       624
change_1h       616
twitter          64
audits           10
description       1
symbol            0
url               0
chain             0
logo              0
name              0
category          0
module            0
slug              0
tvl               0
id                0
dtype: int64

In [83]:
# Convert lists to strings, handling empty or None values
# df['audit_links'] = df['audit_links'].apply(lambda x: str(x) if x is not None and len(x) > 0 else None)
# df['governanceID'] = df['governanceID'].apply(lambda x: str(x) if x is not None and len(x) > 0 else None)
# df['github'] = df['github'].apply(lambda x: str(x) if x is not None and len(x) > 0 else None)

# Now try to save to the database
with connect_db('protocols_tvl') as connection:
    df.to_sql('general', con=connection, if_exists='replace', index=False)


In [86]:
df

,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,parentProtocol,wrongLiquidity,staking,pool2,stablecoins,language,assetToken,oraclesByChain,deadUrl,rugged
0,2269,Binance CEX,None,-,https://www.binance.com,Binance is a cryptocurrency exchange which is ...,Multi-Chain,https://icons.llama.fi/binance-cex.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,182,Lido,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,https://lido.fi/,Liquid staking for Ethereum and Polygon. Daily...,Multi-Chain,https://icons.llama.fi/lido.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2275,Bitfinex,None,-,https://www.bitfinex.com,Bitfinex facilitates a graphical trading exper...,Multi-Chain,https://icons.llama.fi/bitfinex.png,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2272,OKX,None,-,https://www.okx.com,"OKX, formerly known as OKEx, is a Seychelles-b...",Multi-Chain,https://icons.llama.fi/okx.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3732,Robinhood,None,-,https://robinhood.com,Democratizing finance for all. Crypto trading:...,Multi-Chain,https://icons.llama.fi/robinhood.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3574,3707,TokenFi,0x4507cef57c46789ef8d1a19ea45f4216bae2b528,TOKEN,https://tokenfi.com/tokenfi,TokenFi is a platform for crypto and asset tok...,Multi-Chain,https://icons.llama.fi/tokenfi.png,0,None,...,NaN,NaN,6.472883e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3575,3712,Spirit Farm,base:0xC7A148Ad554826b5308E100a05Fdb037DDa0bDe4,SPIRIT,https://www.spiritfarm.app,Spirit Farm a new ecosystem of yield opportuni...,Base,https://icons.llama.fi/spirit-farm.jpg,0,None,...,NaN,NaN,2.724705e-09,3.909703e-10,NaN,NaN,NaN,NaN,True,NaN
3576,3719,AITECH,bsc:0x2D060Ef4d6BF7f9e5edDe373Ab735513c0e4F944,AITECH,https://aitech.io,Solidus has successfully constructed an enviro...,Binance,https://icons.llama.fi/aitech.jpg,2,None,...,NaN,NaN,3.128216e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3577,3761,HOPR,0xf5581dfefd8fb0e4aec526be659cfab1f8c781da,HOPR,https://hoprnet.org,The HOPR protocol provides full control over p...,xDai,https://icons.llama.fi/hopr.jpg,2,None,...,NaN,NaN,6.708313e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
df = pd.DataFrame.from_dict(data, orient = 'Columns')

In [89]:
with connect_db('protocols_tvl') as connection:
    # Create an empty list to hold all the data
    all_data = []

    # Iterate through rows in the DataFrame
    for index, row in df.iterrows():
        name = row['slug']
        chains_data = row['chainTvls']

        # Convert the dictionary of values into a list of dictionaries with 'chain' and 'value' columns
        chains_data_list = [{'chain': chain, 'value': value} for chain, value in chains_data.items()]

        # Add the 'id' column to each dictionary
        for data in chains_data_list:
            data['id'] = name

        # Extend the 'all_data' list with the current data
        all_data.extend(chains_data_list)
        # print(f'{name}')

    # Create a DataFrame from the combined data
    superbig_df = pd.DataFrame(all_data)
    superbig_df['updated_at'] = datetime.now()

    # Insert the combined data into the database as a single table
    table_name = "by_chain"  # Choose a suitable table name
    superbig_df.to_sql(table_name, con=connection, if_exists='replace')

    print(f'All data inserted into {table_name}')

AttributeError: module 'datetime' has no attribute 'now'